In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import random
import cv2

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras import backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.applications import ResNet50, DenseNet121

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


In [3]:
batch_size = 32
img_height = 224
img_width = 224
NUM_CLASSES = 4

In [4]:
DATASET_PATH = r'C:\Users\thanaphat turienngam\Desktop\Cassava\classification\dataset_aug\trainwithaugment'  #ใส่ path Data ที่มี Folder train, test 
Dataname = "ByLT(deleteData)"  #ใส่ชื่อไว้แสดงเฉยๆ

In [5]:
import pathlib
data_dir = DATASET_PATH
data_dir = pathlib.Path(data_dir)

In [6]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

4200


In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=725,
    image_size=(img_height, img_width),
    batch_size=batch_size,label_mode='categorical',
    shuffle = True
)

Found 4200 files belonging to 4 classes.
Using 3360 files for training.


In [8]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=725,
    image_size=(img_height, img_width),
    batch_size=batch_size,label_mode='categorical',
    shuffle = True)

Found 4200 files belonging to 4 classes.
Using 840 files for validation.


In [9]:
class_names = train_ds.class_names
print(class_names)

['cbb', 'cbsd', 'cmd', 'healthy']


In [10]:
# model=tf.keras.applications.DenseNet121(
#     include_top=True,
#     weights=None,
#     input_tensor=None,
#     input_shape=(img_height, img_height, 3),
#     pooling=None,
#     classes=5,
#     classifier_activation="softmax",
# )

In [11]:
# model_name = "DenseNet121"
# optimizer = "Adam"

# model_base=DenseNet121(
#     include_top=False, 
#     input_shape=(img_height, img_height, 3)
# ) 

# x=model_base.output

# x= GlobalAveragePooling2D()(x)
# x= BatchNormalization()(x)
# x= Dropout(0.5)(x)
# x= Dense(1024,activation='relu')(x) 
# x= Dense(512,activation='relu')(x) 
# x= BatchNormalization()(x)
# x= Dropout(0.5)(x)

# preds=Dense(NUM_CLASSES,activation='softmax')(x) #FC-layer

In [12]:
# model_name = "DenseNet121"
# optimizer = "Adam"

# model_base=DenseNet121(
#     include_top=False, 
#     input_shape=(img_height, img_height, 3)
# ) 

# x=model_base.output
# x= layers.Lambda(lambda x, y: (x ,tf.one_hot(y, depth=NUM_CLASSES)))
# x= GlobalAveragePooling2D()(x)
# x= Flatten()
# x= Dense(256,activation='relu')
# x= Dropout(0.7)(x)
# x= BatchNormalization()(x)
# x= Dense(128,activation='relu')(x) 
# x= Dropout(0.5)(x)
# x= BatchNormalization()(x)
# x= Dense(64,activation='relu')(x) 
# x= Dropout(0.3)(x)
# x= BatchNormalization()(x)

# preds=Dense(NUM_CLASSES,activation='softmax')(x) #FC-layer

In [13]:
model_name = "DenseNet121"
optimizer = "Adam"

OldModel=DenseNet121(
    include_top=False, 
    input_shape=(img_height, img_height, 3)
) 


model = Sequential()
# model.add(layers.Lambda(lambda x:K.backend.resize_images(x,height_factor=4,width_factor=4,data_format='channels_last')))
model.add(OldModel)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.7))
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(4, activation='softmax'))

# preds=Dense(NUM_CLASSES,activation='softmax')(x) 

In [14]:
# model=Model(inputs=model_base.input,outputs=preds)

In [15]:
for layer in model.layers[:-NUM_CLASSES]:
    layer.trainable=False
    
for layer in model.layers[-NUM_CLASSES:]:
    layer.trainable=True

In [16]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 batch_normalization (BatchN  (None, 50176)            200704    
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               12845312  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                        

In [17]:
# my_callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=2),
#     tf.keras.callbacks.ModelCheckpoint(filepath='/ch/model.{epoch:02d}-{val_loss:.2f}.h5'),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
# ]

In [18]:
# learningRate = 0.0001
# anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=learningRate)
# checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

In [19]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()

print("now =", now)

# dd/mm/YY H:M:S
str_time = now.strftime("%Hl%Ml%S")
print("date and time =", str_time)

now = 2022-11-25 20:24:42.997176
date and time = 20l24l42


In [ ]:
%%time
MinlearningRateAdj = 0.001
nb_epochs = 120
steps_per_epoch = np.ceil(len(train_ds) / batch_size)
lr = 0.0001

anne = ReduceLROnPlateau(monitor='val_accuracy', 
                         factor=0.5, 
                         patience=2, 
                         min_delta=0.001,
                         verbose=1, 
#                          min_lr=MinlearningRateAdj, 
                        
                        )
CALLBACKS.append(K.callbacks.ModelCheckpoint(
    filepath=r'./ModelSave/modelEnd-'+str_time+'.h5',
    monitor='val_accuracy',
    save_best_only=True))
checkpoint = ModelCheckpoint(r'./ModelSave/modelBest-'+str_time+'.h5', verbose=1, save_best_only=True)


history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=nb_epochs,
    verbose=1,
#     callbacks=[checkpoint]
    callbacks=[anne, checkpoint]
)


Epoch 1/120
105/105 [==============================] - ETA: 0s - loss: 1.9335 - accuracy: 0.2533
Epoch 1: val_loss improved from inf to 1.29930, saving model to ./ModelSave\modelBest-20l24l42.h5
105/105 [==============================] - 69s 459ms/step - loss: 1.9335 - accuracy: 0.2533 - val_loss: 1.2993 - val_accuracy: 0.4262 - lr: 1.0000e-04
Epoch 2/120
105/105 [==============================] - ETA: 0s - loss: 1.7056 - accuracy: 0.3068
Epoch 2: val_loss improved from 1.29930 to 1.16652, saving model to ./ModelSave\modelBest-20l24l42.h5
105/105 [==============================] - 42s 397ms/step - loss: 1.7056 - accuracy: 0.3068 - val_loss: 1.1665 - val_accuracy: 0.5917 - lr: 1.0000e-04
Epoch 3/120
105/105 [==============================] - ETA: 0s - loss: 1.5568 - accuracy: 0.3631
Epoch 3: val_loss improved from 1.16652 to 1.09108, saving model to ./ModelSave\modelBest-20l24l42.h5
105/105 [==============================] - 28s 267ms/step - loss: 1.5568 - accuracy: 0.3631 - val_loss: 1

105/105 [==============================] - 30s 283ms/step - loss: 1.2582 - accuracy: 0.4902 - val_loss: 0.9999 - val_accuracy: 0.5940 - lr: 1.9531e-07
Epoch 24/120
105/105 [==============================] - ETA: 0s - loss: 1.2616 - accuracy: 0.4848
Epoch 24: ReduceLROnPlateau reducing learning rate to 9.765624753299562e-08.

Epoch 24: val_loss did not improve from 0.99986
105/105 [==============================] - 29s 270ms/step - loss: 1.2616 - accuracy: 0.4848 - val_loss: 1.0003 - val_accuracy: 0.5952 - lr: 1.9531e-07
Epoch 25/120
105/105 [==============================] - ETA: 0s - loss: 1.2440 - accuracy: 0.5080
Epoch 25: val_loss did not improve from 0.99986
105/105 [==============================] - 31s 288ms/step - loss: 1.2440 - accuracy: 0.5080 - val_loss: 0.9999 - val_accuracy: 0.5952 - lr: 9.7656e-08
Epoch 26/120
105/105 [==============================] - ETA: 0s - loss: 1.2452 - accuracy: 0.5015
Epoch 26: ReduceLROnPlateau reducing learning rate to 4.882812376649781e-08.

E

Epoch 48/120
105/105 [==============================] - ETA: 0s - loss: 1.2391 - accuracy: 0.5077
Epoch 48: ReduceLROnPlateau reducing learning rate to 2.384185730786026e-11.

Epoch 48: val_loss did not improve from 0.99957
105/105 [==============================] - 27s 253ms/step - loss: 1.2391 - accuracy: 0.5077 - val_loss: 1.0015 - val_accuracy: 0.5952 - lr: 4.7684e-11
Epoch 49/120
105/105 [==============================] - ETA: 0s - loss: 1.2412 - accuracy: 0.5024
Epoch 49: val_loss did not improve from 0.99957
105/105 [==============================] - 28s 260ms/step - loss: 1.2412 - accuracy: 0.5024 - val_loss: 1.0010 - val_accuracy: 0.5952 - lr: 2.3842e-11
Epoch 50/120
105/105 [==============================] - ETA: 0s - loss: 1.2365 - accuracy: 0.5104
Epoch 50: ReduceLROnPlateau reducing learning rate to 1.192092865393013e-11.

Epoch 50: val_loss did not improve from 0.99957
105/105 [==============================] - 29s 275ms/step - loss: 1.2365 - accuracy: 0.5104 - val_loss: 

In [ ]:
model.save(r'./ModelSave/modelEnd-'+str_time+'.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(nb_epochs)

print("Model = "+model_name)
print("DataName = "+Dataname)
print("Optimizer = "+optimizer)
print("Epochs = {}".format(nb_epochs))
print("Image Size = {}".format(img_width))
print("Batch = {}".format(batch_size))
print("learningRate = {}".format(lr))



plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

#####  use this  to save train graph as png #####################
from datetime import datetime
dt = datetime.now()
ts = datetime.timestamp(dt)
date_time = datetime.fromtimestamp(ts)
str_date_time = date_time.strftime("%d-%m-%Y_%H-%M")
graph_path = r'C:\Users\thanaphat turienngam\Desktop\Cassava\classification\experimentData\graph'
if( not (os.path.exists(graph_path)) ) :
    os.mkdir(graph_path)
plt.savefig(graph_path+'/'+str_date_time+".png")  
#####  use this  to save train graph as png #####################

plt.show()


graph_path = graph_path+'/'+str_date_time+".png"


In [ ]:
#model = keras.models.load_model(r"C:\Users\Chale\Desktop\AI WORK\CODE\cassava_model\model\DenseNet121noaug94.h5")

In [ ]:
model.summary()

In [ ]:
# img_path = DATASET_PATH+r"\test\cmd\test-img-43.jpg"
# img = tf.keras.utils.load_img(
#     img_path, target_size=(img_height, img_width)
# )
# img_array = tf.keras.utils.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0) # Create a batch

# predictions = model.predict(img_array)
# score = tf.nn.softmax(predictions[0])

# print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(class_names[np.argmax(score)], 100 * np.max(score))
# )

In [ ]:
test_dir =r"C:\Users\thanaphat turienngam\Desktop\Cassava\classification\dataset_aug\trainwithaugment\test"
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=1,
    shuffle = True
)

In [ ]:
test_data = test_ds.map(tf.autograph.experimental.do_not_convert(lambda x, y: (x , 
                                      tf.one_hot(y, depth=NUM_CLASSES))))
result = model.evaluate(test_data)

In [ ]:
test_x = []
test_y = []
for images, labels in test_data:
    test_x.append(images)
    test_y.append(np.argmax(labels))

In [ ]:
%%time
preds = []
i=0;
for img in test_x:
    preds.append(np.argmax(model.predict(img, verbose=0)))
    i=i+1
    print("Running : "+str(i)+"/"+str(len(test_x))+" = "+str(i/len(test_x)*100)+"%")

In [ ]:
preds = np.array(preds)
test_y = np.array(test_y)

In [ ]:
print(preds)
print(test_y)

In [ ]:
class_names = test_ds.class_names

In [ ]:
confm = tf.math.confusion_matrix(
    test_y,
    preds,
    num_classes=NUM_CLASSES,
    dtype=tf.dtypes.int32,
    name=class_names
)

In [ ]:
confm = np.array(confm)

In [ ]:
import seaborn as sns
import pandas as pd

df_cm = pd.DataFrame((confm/confm.astype(np.float32).sum(axis=1)), index =class_names,
              columns =class_names )
plt.figure(figsize = (12,9))
plt.title('All fold confusion matrix of the multi-class classification\n\nModel '+model_name+'\n\nMean Accuracy is {:.2f}'.format((np.mean(result[1]))))
sns.heatmap(df_cm, annot=True,cmap="Blues",fmt=".2f")